In [ ]:
import os
import pandas as pd

In [ ]:
from {{package_name}}.models_training.model_tfidf_svm import ModelTfidfSvm 

In [ ]:
def text_to_sentence(text, min_sentence_size, min_sentence_word):
    text = re.sub(r'\s',' ', text)
    text = re.sub(r'\!', r'.', text)
    text = re.sub(r'\?', r'.', text)
    text = re.sub(' +', ' ', text)
    list_sentences = text.split('.')
    list_sentences = [sentence for sentence in list_sentences if len(sentence) >= min_sentence_size]
    list_sentences = [sentence for sentence in list_sentences if len(sentence.split(' ')) >= min_sentence_word]
    return list_sentences

In [ ]:
min_sentence_size = 50
min_sentence_word = 10
df_texts = pd.read_csv('../{{package_name}}-data/texts.csv', sep='|')

In [ ]:
list_phrases = []
for index, row in df_texts.iterrows():
    text = row['text']
    author = row['author']
    book = row['book']
    sentences = text_to_sentence(text, min_sentence_size, min_sentence_word)
    list_phrases = list_phrases+[(sentence, author, book) for sentence in sentences]
df_phrases = pd.DataFrame(list_phrases, columns=['sentence', 'author', 'book'])
set_author = set(df_texts['author'])
df_phrases.value_counts('author')/len(df_phrases)

In [ ]:
dict_books_to_test = {}
for author in set_author:
    set_books = set(df_texts[df_texts['author']==author]['book'])
    dict_books_to_test[author] = random.sample(set_books, k=1)[0]
df_test = df_phrases[df_phrases['book'].isin(dict_books_to_test.values())].copy()

df = df_phrases.sample(frac=1)
df = df[~df['book'].isin(dict_books_to_test.values())]
df_train = df[:int(4*len(df)/4)].copy()
df_valid = df[int(4*len(df)/4):].copy()

dict_books_to_test

In [ ]:
dict_result = {}
count =0
for ngram_range in [(1, 1), (1, 2)]:
    for C in [0.5, 1, 2]:
        model = ModelTfidfSvm(tfidf_params = {'ngram_range': ngram_range}, svc_params={'C':C})
        model.fit(df_train['sentence'], df_train['author'], x_valid=df_valid['sentence'], y_valid=df_valid['author'])
        df_valid['pred'] = model.predict(df_valid['sentence'])
        score = f1_score(df_valid['author'], df_valid['pred'], average='macro')
        dict_tmp = {'score':score, 'ngram_range':ngram_range, 'C':C}
        dict_result[count] = dict_tmp.copy()
        count += 1
        print(dict_tmp)

In [ ]:
model = ModelTfidfSvm(tfidf_params = {'ngram_range': (1, 2)}, svc_params={'C':1})
model.fit(df_train['sentence'], df_train['author'], x_valid=df_valid['sentence'], y_valid=df_valid['author'])

In [ ]:
df_test['pred'] = model.predict(df_test['sentence'])
score = f1_score(df_test['author'], df_test['pred'], average='macro')
print(score)

In [ ]:
def text_to_dataframe(text, min_sentence_size, min_sentence_word, author=None, book=None):
    sentences = text_to_sentence(text, min_sentence_size, min_sentence_word)
    sentences = [(sentence, author, book) for sentence in sentences]
    df = pd.DataFrame(sentences, columns=['sentence', 'author', 'book'])
    return df

def predict_author(text, model, min_sentence_size, min_sentence_word):
    df = text_to_dataframe(text, min_sentence_size, min_sentence_word)
    counter = dict(Counter(list(model.predict(df['sentence']))))
    max_value = max(counter, key=counter.get)
    return max_value, counter

def take_random_part_text(text, length):
    text_length = len(text)
    beginning = random.randrange(0, (text_length-length)-1)
    return text[beginning: beginning+length]

In [ ]:
dict_result = {}
random_part = False
for author, book in dict_books_to_test.items():
    row = df_texts[df_texts['book']==book]
    text = row.iloc[0]['text']
    length = len(text)
    if random_part:
        text = take_random_part_text(text, min_sentence_size*300)
    prediction, counter = predict_author(text, model, min_sentence_size, min_sentence_word)
    dict_result[author] = {'prediction': prediction, 'counter': counter, 'length': len(text), 'perc_text':len(text)/length}
dict_result